In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd

In [2]:
config = configparser.RawConfigParser()
config.read(r'C:\Users\m088378\Documents\myKeyVariables.cfg')   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False
outDir = envVars['pointsdownloaded']

conn = ezomero.connect(user, password, 'Prakash.YS_Lab', host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	2024_Panel:	903
	20240119_YSPTRNeo2_Prakash:	905
	20220415_YSP_CODEX_Hs_QC_Prakash:	301
	20230516_req33419COPD_Prakash:	918
	20221021_Req33382_Prakash:	353
	20230201_YSPTRneo1_Prakash:	482
	20230407_Req33419COPD_Prakash:	618
	20230407_Req32370HS_Prakash:	803
	20230505_req32370HS_Prakash:	708
	20230512_req33421ASTHAM_Prakash:	711
	Manuscript:	910


In [3]:
allSamps = ezomero.get_dataset_ids(conn, project=918)
allSamps.extend( ezomero.get_dataset_ids(conn, project=708) )
allSamps.extend( ezomero.get_dataset_ids(conn, project=711) )
print("There are {} datasets to investigate.".format(len(allSamps)))

There are 3 datasets to investigate.


In [4]:
## Image Desc
## https://docs.openmicroscopy.org/omero/5.4.1/developers/PythonBlitzGateway.html
# https://downloads.openmicroscopy.org/omero/5.4.10/api/python/omero/omero.gateway.html#omero.gateway._ImageWrapper
fout = open(os.path.join(outDir,'log_omero_points.txt'), 'w')
for sm in allSamps:
    directory_path = os.path.join(outDir,"Dataset_"+str(sm))
    # Check if the directory exists
    if not os.path.exists(directory_path):
        # Create the directory
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created successfully.")
    
    imgs = ezomero.get_image_ids(conn, dataset=sm)
    for im in imgs:
        imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
        if(imgDesc.getROICount(shapeType="Point") > 0):
            fout.write(imgDesc.getName()+'\n')
            shName = imgDesc.getName().replace(".ome.tiff",'')
            imgPointsIDs = ezomero.get_roi_ids(conn, im)
            print("{} has {}".format(imgDesc.getName().replace('\\.ome\\.tiff',''), len(imgPointsIDs)))
            fout.write("{} Annotations\n".format(len(imgPointsIDs)))
            allThesePoints = []
            for i in imgPointsIDs:
                for ppI in ezomero.get_shape_ids(conn, i):
                    try:
                        pt, fill, border, width = ezomero.get_shape(conn, ppI)
                        allThesePoints.append([pt.x, pt.y, pt.label,pt.label, fill, border, width])
                    except:
                        print("   BAD point ID : "+str(i))
            #pprint(allThesePoints)
            pointTable = pd.DataFrame(allThesePoints)
            pointTable.columns = ["x","y","class", "name","fill","border","width"]
            cntTbl = pointTable['class'].value_counts().to_frame()
            noms = cntTbl.index.to_list()
            cnts = cntTbl.values
            for n in range(len(noms)):
                fout.write("{}\t{}\n".format(noms[n],cnts[n][0]))
            fout.write("\n")
            pointTable.to_csv(os.path.join(directory_path,shName+'-points.tsv'), index=False, sep="\t")


reg002.ome.tiff has 115
reg003.ome.tiff has 123
reg004.ome.tiff has 116
A-1.ome.tif has 85
A-2.ome.tif has 91
A-3.ome.tif has 116
A-1.ome.tif has 52
A-2.ome.tif has 53
A-3.ome.tif has 55


In [5]:
fout.close()